In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from holsteinlib import windowing, functions, feature_functions
from holsteinlib.feature_functions import calc_magnitude, calculate_VeDBA, calculate_ODBA, calculate_pitch, calculate_roll

In [ ]:
dataset_df = pd.read_csv(functions.return_holstein_dataset_path('Thesis Versions'))

In [ ]:
dataset_df.head()

In [6]:
seg_ids = dataset_df.segId.unique()

In [7]:
combined_df = pd.DataFrame()

In [ ]:
# Magnitude TS
dataset_df['adjMag'] = dataset_df.apply(lambda row: calc_magnitude(row['accX'], row['accY'], row['accZ']), axis=1)

In [8]:
for sId in seg_ids:
    seg_df = dataset_df[dataset_df.segId == sId].copy()
    
    # static component
    sos_static = signal.butter(6, 0.3, 'low', output = 'sos', analog=False)
    seg_df['static_x'] = signal.sosfilt(sos_static, seg_df['accX'])
    seg_df['static_y'] = signal.sosfilt(sos_static, seg_df['accY'])
    seg_df['static_z'] = signal.sosfilt(sos_static, seg_df['accZ'])
    
    # Dinamic component
    sos_dinamic = signal.butter(6, 0.3, 'high', output = 'sos', analog=False)
    seg_df['dynamic_x'] = signal.sosfilt(sos_dinamic, seg_df['accX'])
    seg_df['dynamic_y'] = signal.sosfilt(sos_dinamic, seg_df['accY'])
    seg_df['dynamic_z'] = signal.sosfilt(sos_dinamic, seg_df['accZ'])
    
    # Calculate ODBA and VeDBA from dynamic componets
    seg_df['VeDBA'] = seg_df.apply(lambda x: calculate_VeDBA(x.dynamic_x, x.dynamic_y, x.dynamic_z), axis =1)
    seg_df['ODBA'] = seg_df.apply(lambda x: calculate_ODBA(x.dynamic_x, x.dynamic_y, x.dynamic_z), axis =1)

    # Calculate pith and roll from static componets
    seg_df['pitch'] = seg_df.apply(lambda x: calculate_pitch(x.static_x,x.static_y,x.static_z), axis =1)
    seg_df['roll'] = seg_df.apply(lambda x: calculate_roll(x.static_x,x.static_y,x.static_z), axis =1)
    
    combined_df = pd.concat([combined_df, seg_df])

In [9]:
combined_df = combined_df.sort_values(by=['segId', 'dateTime'])
combined_df.reset_index(drop=True, inplace=True)

In [10]:
combined_df_ = combined_df[['dateTime', 'calfId', 'segId', 
                            'accX', 'accY', 'accZ', 'adjMag', 'ODBA', 'VeDBA', 'pitch', 'roll', 
                            'behaviour']]

In [11]:
combined_df_

,dateTime,calfId,segId,accX,accY,accZ,adjMag,ODBA,VeDBA,pitch,roll,behaviour
0,2022-02-24 00:08:34.402385,1306,0,0.812500,0.390625,0.281250,0.0,0.219290,0.139515,17.326459,24.433259,oral_manipulation_of_pen
1,2022-02-24 00:08:34.441385,1306,0,0.687500,0.375000,0.375000,0.0,0.581527,0.378783,18.143089,24.573772,oral_manipulation_of_pen
2,2022-02-24 00:08:34.481386,1306,0,0.625000,0.296875,0.265625,0.0,0.202425,0.149251,18.969206,24.682772,oral_manipulation_of_pen
3,2022-02-24 00:08:34.521386,1306,0,0.703125,0.218750,0.218750,0.0,0.459719,0.279852,19.740176,24.691324,oral_manipulation_of_pen
4,2022-02-24 00:08:34.560387,1306,0,0.734375,0.218750,0.250000,0.0,0.107439,0.063745,20.361440,24.480466,oral_manipulation_of_pen
...,...,...,...,...,...,...,...,...,...,...,...,...
2467157,2022-02-22 20:29:29.977497,1449,4015,0.734375,-0.578125,0.296875,0.0,0.026215,0.015815,17.608833,-36.523770,eating_bedding
2467158,2022-02-22 20:29:30.019498,1449,4015,0.734375,-0.578125,0.296875,0.0,0.009803,0.005938,17.438276,-36.591767,eating_bedding
2467159,2022-02-22 20:29:30.060499,1449,4015,0.718750,-0.546875,0.281250,0.0,0.005925,0.003990,17.516419,-36.421119,eating_bedding
2467160,2022-02-22 20:29:30.101501,1449,4015,0.718750,-0.578125,0.296875,0.0,0.034301,0.023622,17.696488,-36.133211,eating_bedding


In [12]:
combined_df_.to_csv('Datasets/holstein_dataset_thesis_extra_TS_v1.csv', index=False)